In [1]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.10/index.html
!git clone https://github.com/open-mmlab/mmsegmentation.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.10/index.html
     |████████████████████████████████| 48.5 MB 11.0 MB/s 
     |████████████████████████████████| 190 kB 5.1 MB/s 
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 7404, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 7404 (delta 60), reused 100 (delta 53), pack-reused 7276
Receiving objects: 100% (7404/7404), 13.87 MiB | 37.78 MiB/s, done.
Resolving deltas: 100% (5450/5450), done.


In [2]:
import torch

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0+cu113 (Tesla P100-PCIE-16GB)


In [3]:
%cd mmsegmentation

/content/mmsegmentation


In [4]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
import mmcv

In [5]:
import os.path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [6]:
# %cd checkpoints
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

classes = (
        'wall', 'building', 'sky', 'floor', 'tree', 'ceiling', 'road', 'bed ',
        'windowpane', 'grass', 'cabinet', 'sidewalk', 'person', 'earth',
        'door', 'table', 'mountain', 'plant', 'curtain', 'chair', 'car',
        'water', 'painting', 'sofa', 'shelf', 'house', 'sea', 'mirror', 'rug',
        'field', 'armchair', 'seat', 'fence', 'desk', 'rock', 'wardrobe',
        'lamp', 'bathtub', 'railing', 'cushion', 'base', 'box', 'column',
        'signboard', 'chest of drawers', 'counter', 'sand', 'sink',
        'skyscraper', 'fireplace', 'refrigerator', 'grandstand', 'path',
        'stairs', 'runway', 'case', 'pool table', 'pillow', 'screen door',
        'stairway', 'river', 'bridge', 'bookcase', 'blind', 'coffee table',
        'toilet', 'flower', 'book', 'hill', 'bench', 'countertop', 'stove',
        'palm', 'kitchen island', 'computer', 'swivel chair', 'boat', 'bar',
        'arcade machine', 'hovel', 'bus', 'towel', 'light', 'truck', 'tower',
        'chandelier', 'awning', 'streetlight', 'booth', 'television receiver',
        'airplane', 'dirt track', 'apparel', 'pole', 'land', 'bannister',
        'escalator', 'ottoman', 'bottle', 'buffet', 'poster', 'stage', 'van',
        'ship', 'fountain', 'conveyer belt', 'canopy', 'washer', 'plaything',
        'swimming pool', 'stool', 'barrel', 'basket', 'waterfall', 'tent',
        'bag', 'minibike', 'cradle', 'oven', 'ball', 'food', 'step', 'tank',
        'trade name', 'microwave', 'pot', 'animal', 'bicycle', 'lake',
        'dishwasher', 'screen', 'blanket', 'sculpture', 'hood', 'sconce',
        'vase', 'traffic light', 'tray', 'ashcan', 'fan', 'pier', 'crt screen',
        'plate', 'monitor', 'bulletin board', 'shower', 'radiator', 'glass',
        'clock', 'flag')

palette = [[120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]]
# classes = ('background', 'truck')
# palette = [[0,0,0], [255,0,0]]
classes = ['background', 'container_truck' , 'forklift' , 'reach_stacker', 'ship']
palette = [[0, 0, 0], [0, 0, 255], [0, 255, 0], [255, 0, 0], [255, 0, 255]]
@DATASETS.register_module()
class SIADataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

In [8]:
# tiny
#config_file = '/content/mmsegmentation/configs/setr/setr_vit-large_naive_8x1_768x768_80k_cityscapes.py'
#config_file = "/content/mmsegmentation/configs/convnext/upernet_convnext_large_fp16_640x640_160k_ade20k.py"
config_file = '/content/mmsegmentation/configs/convnext/upernet_convnext_tiny_fp16_512x512_160k_ade20k.py'
# checkpoint_file = '/content/drive/MyDrive/SIA/semantic_checkpoint/upernet_convnext_large_fp16_640x640_160k_ade20k_20220226_040532-e57aa54d.pth'
# base
# config_file = "/content/mmsegmentation/configs/convnext/upernet_convnext_base_fp16_640x640_160k_ade20k.py" 
# checkpoint_file = '/content/drive/MyDrive/SIA/semantic_checkpoint/upernet_convnext_base_fp16_640x640_160k_ade20k_20220227_182859-9280e39b.pth'
from mmcv import Config
!pip install mmcls
cfg = Config.fromfile(config_file)
# print(cfg.pretty_text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 577 kB 5.0 MB/s 


In [9]:
from mmseg.apis import set_random_seed

# Since we use ony one GPU, BN is used instead of SyncBN
# cfg.norm_cfg = dict(type='BN', requires_grad=True)

# cfg.model.backbone.norm_cfg = cfg.norm_cfg
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 5
cfg.model.auxiliary_head.num_classes = 5

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2
# 전체 train img
# cfg.img_norm_cfg = dict(
#     mean=[117.871, 115.512, 118.21], std=[45.079, 45.176, 42.465], to_rgb=True)
# truck 29개 짜리
cfg.img_norm_cfg = dict(
    mean=[92.628, 93.721, 99.875], std=[12.688, 12.548, 10.482], to_rgb=True)
cfg.crop_size = (256, 256)
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    dict(type='Resize', img_scale=(512, 256), ratio_range=(0.5, 1.0)),
    #dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    #dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    #dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512, 256),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.dataset_type = 'SIADataset'
cfg.data_root = '/content/drive/MyDrive/Challenge_AI'

cfg.data.train.type = 'SIADataset'
cfg.data.train.data_root = '/content/drive/MyDrive/Challenge_AI'
cfg.data.train.img_dir = 'images'
cfg.data.train.ann_dir = 'label'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/challenge_ai.txt'

cfg.data.val.type = 'SIADataset'
cfg.data.val.data_root = '/content/drive/MyDrive/Challenge_AI'
cfg.data.val.img_dir = 'images'
cfg.data.val.ann_dir = 'label'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/challenge_ai.txt'

cfg.data.test.type = 'SIADataset'
cfg.data.test.data_root = '/content/drive/MyDrive/Challenge_AI'
cfg.data.test.img_dir = 'images'
cfg.data.test.ann_dir = 'label'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/challenge_ai.txt'

# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = '/content/drive/MyDrive/model/upernet_convnext_tiny_fp16_512x512_160k_ade20k_20220227_124553-cad485de.pth'

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/Challenge_AI'

cfg.runner.max_iteTrs = 200
cfg.log_config = log_config = dict(  # config to register logger hook
    interval=50,  # Interval to print the log
    hooks=[
        # dict(type='TensorboardLoggerHook')  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook', by_epoch=False)
    ])

cfg.evaluation.interval = 1000
cfg.checkpoint_config.interval = 1000

cfg.runner = dict(type='IterBasedRunner', max_iters=1000)

# cfg.workflow = [('train', 2)]

# Set seed to facitate reproducing the result
cfg.device = 'cuda'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

In [10]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

2022-07-01 07:25:16,366 - mmseg - INFO - Loaded 135 images


In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [12]:
# Build the detector
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

train_segmentor(model, datasets, cfg, distributed=False, validate=True, meta=dict(CLASSES=classes, PALETTE=palette))

/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
2022-07-01 07:25:30,618 - mmseg - INFO - self.paramwise_cfg is {'decay_rate': 0.9, 'decay_type': 'stage_wise', 'num_layers': 6}
2022-07-01 07:25:30,620 - mmseg - INFO - Build LearningRateDecayOptimizerConstructor  stage_wise 0.9 - 8
2022-07-01 07:25:30,622 - mmseg - INFO - set param backbone.downsample_layers.0.0.weight as id 0
2022-07-01 07:25:30,624 - mmseg - INFO - set param backbone.downsample_layers.0.0.bias as id 0
2022-07-01 07:25:30,626 - mmseg - INFO - set param backbone.downsample_layers.0.1.weight as id 0
2022-07-01 07:25:30,629 - mmseg - INFO - set param backbone.downsample_layers.0.1.bias as id 0
2022-07-01 07:25:30,630 - mm

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 135/135, 8.2 task/s, elapsed: 17s, ETA:     0s

2022-07-01 07:31:05,799 - mmseg - INFO - per class results:
2022-07-01 07:31:05,801 - mmseg - INFO - 
+-----------------+-------+-------+
|      Class      |  IoU  |  Acc  |
+-----------------+-------+-------+
|    background   | 96.06 |  97.5 |
| container_truck | 80.05 | 91.85 |
|     forklift    | 76.78 | 89.84 |
|  reach_stacker  | 65.07 | 84.25 |
|       ship      |  74.0 |  81.5 |
+-----------------+-------+-------+
2022-07-01 07:31:05,803 - mmseg - INFO - Summary:
2022-07-01 07:31:05,805 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 96.47 | 78.39 | 88.98 |
+-------+-------+-------+
2022-07-01 07:31:05,808 - mmseg - INFO - Iter(val) [135]	aAcc: 0.9647, mIoU: 0.7839, mAcc: 0.8898, IoU.background: 0.9606, IoU.container_truck: 0.8005, IoU.forklift: 0.7678, IoU.reach_stacker: 0.6507, IoU.ship: 0.7400, Acc.background: 0.9750, Acc.container_truck: 0.9185, Acc.forklift: 0.8984, Acc.reach_stacker: 0.8425, Acc.ship: 0.8150


In [ ]:
import os
import cv2
a = os.listdir('/content/drive/MyDrive/SIA/pre_labels')
for i in a:
  img = cv2.imread(f'/content/drive/MyDrive/SIA/pre_labels/{i}')
  print(img.shape)

In [ ]:
import os
import cv2

def make_predata():
  a = os.listdir('/content/drive/MyDrive/SIA/labels')
  for name in a:
    img = cv2.imread(f'/content/drive/MyDrive/SIA/labels/{name}', cv2.IMREAD_GRAYSCALE)
    if name[0] == 'c': number = 83
    elif name[0] == 'f': number = 80
    elif name[0] == 'r': number = 122
    elif name[0] == 's': number = 76
    
    img = np.where(img != 0, number, 0)
    cv2.imwrite(f'/content/drive/MyDrive/SIA/pre_labels2/{name}', img)
make_predata()

In [ ]:
import os
import cv2

def make_predata():
  a = os.listdir('/content/drive/MyDrive/SIA/labels')
  for name in a:
    img = cv2.imread(f'/content/drive/MyDrive/SIA/labels/{name}', cv2.IMREAD_GRAYSCALE)
    if name[0] == 'c': number = 83
    elif name[0] == 'f': number = 80
    elif name[0] == 'r': number = 122
    elif name[0] == 's': number = 76
    
    img = np.where(img != 0, number, 0)
    print(img.shape)
    break
    cv2.imwrite(f'/content/drive/MyDrive/SIA/pre_labels2/{name}', img)
make_predata()

In [ ]:
import os
import cv2
a = os.listdir('/content/drive/MyDrive/SIA/before_pre')
for i in a:
  img = cv2.imread(f'/content/drive/MyDrive/SIA/before_pre/{i}', cv2.IMREAD_UNCHANGED)
  print(img.shape)
  cv2.imwrite(f'/content/drive/MyDrive/SIA/pre_labels/{i}', img)